# Eda

## Dependencias

In [1]:
# data wrangling
import pandas as pd
import numpy as np
import unicodedata
import re
import datetime

# Visualizacion
import seaborn as sb
import matplotlib.pyplot as plt
from plotly.offline import plot,iplot
import plotly.express as px
import plotly.graph_objects as go

# Display options
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 35)

In [2]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"}) 
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

In [3]:
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    text=str(text)
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().lstrip().split())
    return cleaned_text if cleaned_text!="nan" else np.nan

## Lectura de datos

In [4]:
df=pd.read_csv('../data/processed/df_data_quality',error_bad_lines=False,low_memory=False)
df.head(3)

,d_datetime,t_city,v_state,v_country,v_shape,c_duration(seconds),t_duration(hours/min),t_comments,d_dateposted,c_latitude,c_longitude
0,1949-10-10,san marcos,tx,us,cylinder,2700.0,45 minutes,this event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10,lackland afb,tx,NaN,light,7200.0,1 2 hrs,1949 lackland afb 44 tx lights racing across t...,2005-12-16,29.384210,-98.581082
2,1955-10-10,chester uk england,NaN,gb,circle,20.0,20 seconds,green orange circular disc over chester 44 eng...,2008-01-21,53.200000,-2.916667
